In [2]:
from music21 import converter, instrument, note, stream, chord
import pickle
import glob
import numpy as np
from keras.utils import np_utils
from keras.models import load_model

## Read a Midi File

In [3]:
midi = converter.parse('./midi_songs/EyesOnMePiano.mid')

In [4]:
midi

<music21.stream.Score 0x7fadcf601d00>

In [5]:
midi.show('midi')

In [6]:
midi.show('text')

{0.0} <music21.stream.Part 0x7fadcf601b20>
    {0.0} <music21.instrument.Instrument ''>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=95.0>
    {0.0} <music21.key.Key of D major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x7fadd79ae700>
        {0.0} <music21.note.Note A>
        {0.0} <music21.note.Note A>
        {0.6667} <music21.note.Note G>
        {1.5} <music21.note.Note D>
        {2.0} <music21.note.Note A>
        {2.5} <music21.note.Note G>
        {3.0} <music21.note.Note D>
        {3.5} <music21.note.Note A>
        {4.0} <music21.note.Note A>
        {6.0} <music21.note.Note A>
        {8.0} <music21.note.Note F#>
        {10.0} <music21.chord.Chord F#4 C#5>
        {11.0} <music21.note.Note A>
        {11.6667} <music21.note.Note D>
        {13.3333} <music21.note.Note D>
        {13.6667} <music21.note.Note C#>
        {14.0} <music21.note.Note D>
        {15.0} <music21.note.Note D>
      

In [7]:
elements_to_parse = midi.flat.notes

In [8]:
len(elements_to_parse)

1421

In [9]:
# for e in elements_to_parse:
#     print(e," offset --> ",e.offset)  # offset --> at what time the note or chord is played

# A3 --> Note of 3rd octave...


In [10]:
def make_notes(elements_to_parse):
    notes_demo = []

    for ele in elements_to_parse:
        # If the elemet is a note, then store it's pitch
        if isinstance(ele,note.Note):
            notes_demo.append(str(ele.pitch))
        # If the element is a chord, split each note of chord and join them with '+'
        elif isinstance(ele,chord.Chord):
            notes_demo.append("+".join(str(n) for n in ele.normalOrder ))
    
    return notes_demo

In [11]:
str(elements_to_parse[3].pitch)

'A6'

In [12]:
"+".join(str(n) for n in elements_to_parse[68].normalOrder)

'1+6'

In [13]:
isinstance(elements_to_parse[68],chord.Chord)


True

In [14]:
len(make_notes(elements_to_parse))

1421

### Preprocessing all Files

In [15]:
notes = []

for file in glob.glob('midi_songs/*.mid'):
    try:
        midi = converter.parse(file) # Convert file into stream.Score object
        print("parsing %s"%file)
        elements_to_parse = midi.flat.notes
        notes.extend(make_notes(elements_to_parse))
    except:
        print("Parse error --> %s"%file)
    

parsing midi_songs/Ff7-Jenova_Absolute.mid
parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
parsing midi_songs/ahead_on_our_way_piano.mid
parsing midi_songs/balamb.mid
parsing midi_songs/DOS.mid
parsing midi_songs/FF3_Battle_(Piano).mid
parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
parsing midi_songs/caitsith.mid
parsing midi_songs/sera_.mid
parsing midi_songs/Oppressed.mid
parsing midi_songs/ff4-airship.mid
parsing midi_songs/FFVII_BATTLE.mid
parsing midi_songs/electric_de_chocobo.mid
parsing midi_songs/Ff7-One_Winged.mid
parsing midi_songs/Finalfantasy5gilgameshp.mid
parsing midi_songs/Cids.mid
parsing midi_songs/lurk_in_dark.mid
parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
parsing midi_songs/ff4pclov.mid
parsing midi_songs/AT.mid
parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
parsing midi_songs/8.mid
parsing midi_songs/sobf.mid
parsing midi_songs/In_Zanarkand.mid
parsing midi_songs/waltz_de_choco.mid
parsing midi_songs/z_aeristhemepiano.

In [16]:
# notes

In [17]:
len(notes)

59652

In [18]:
with open('notes','wb') as filepath:
    pickle.dump(notes,filepath)
    

In [19]:
with open('notes','rb') as f:
    notes = pickle.load(f)
    

In [20]:
n_vocab = len(set(notes))

In [21]:
print("Total Notes -->",len(notes))
print('Unique Notes -->',n_vocab)

Total Notes --> 59652
Unique Notes --> 358


### Prepare Sequential Data for LSTM

In [22]:
# How many elements LSTM input should consider
sequence_length = 100

In [23]:
pitchname = sorted(set(notes))

In [24]:
# Mapping between ele into int value
ele_to_int = dict((ele, num) for num, ele in enumerate(pitchname))

In [25]:
network_input = []
network_output = []

for i in range(len(notes)- sequence_length):
    seq_in = notes[i :i+sequence_length]  # contains 100 values
    seq_out = notes[i + sequence_length] # 101th value after every 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])

In [26]:
# network_output
n_patterns = len(network_input)
print(n_patterns)

59552


In [27]:
# network_output

In [28]:
network_input = np.reshape(network_input, (n_patterns,sequence_length,1))

In [29]:
normalized_network_input = network_input//float(n_vocab)

In [30]:
# Network output are the classes, encode into one hot vector


In [31]:
network_output = np_utils.to_categorical(network_output)

In [32]:
network_output.shape

(59552, 358)

In [33]:
print(normalized_network_input.shape)
print(network_output.shape)

(59552, 100, 1)
(59552, 358)


## Create Model

In [34]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [35]:
model = Sequential()
model.add(LSTM(512,input_shape=(normalized_network_input.shape[1],normalized_network_input.shape[2]),return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences= True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation='softmax'))

In [36]:
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
dropout (Dropout)            (None, 100, 512)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [107]:
checkpoint = ModelCheckpoint('model.hdf5',monitor='loss',verbose=0,save_best_only=True,mode='min')
model_his = model.fit(normalized_network_input,network_output,epochs=100,batch_size=64,callbacks=[checkpoint])


In [41]:
model = load_model('new_weights.hdf5')

### Predictions

In [42]:
sequence_length = 100
network_input = []
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i + sequence_length] # contains 100 value
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [43]:
start = np.random.randint(len(network_input) - 1)
start

13347

In [44]:
# Mapping int_to_ele

In [45]:
int_to_ele = dict((num,ele) for num,ele in enumerate(pitchname))

In [51]:
# Initial Pattern
pattern = network_input[start]
prediction_output = []

# generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    
    prediction_output.append(result)
    
    pattern.append(idx)
    pattern = pattern[1:]
                    
    

In [53]:
# prediction_output

### Create Midi file

In [61]:
offset = 0 # Time 
output_notes = []
for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = [] 
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) # create Note object for each note in the chord 
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes)  # creates the chord) from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    # if the pattern is a Note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    
    offset += 0.5

In [54]:
prediction_output[4]

'G#5'

In [59]:
# new_note = note.Note(prediction_output[4])
# new_note.offset = 0
# new_note.storedInstrument = instrument.Piano()
# output_notes.append(new_note)

In [62]:
output_notes

[<music21.chord.Chord G# C>,
 <music21.chord.Chord E- E>,
 <music21.chord.Chord E- F# B->,
 <music21.note.Note G#>,
 <music21.note.Note G#>,
 <music21.chord.Chord G# B C#>,
 <music21.note.Note E->,
 <music21.note.Note G#>,
 <music21.note.Note B->,
 <music21.chord.Chord E F# G# B C#>,
 <music21.note.Note B>,
 <music21.chord.Chord F# B->,
 <music21.chord.Chord E- E>,
 <music21.note.Note B>,
 <music21.note.Note B->,
 <music21.note.Note F#>,
 <music21.note.Note E->,
 <music21.note.Note F#>,
 <music21.note.Note F#>,
 <music21.chord.Chord F# G# C>,
 <music21.chord.Chord F# G# C>,
 <music21.chord.Chord B- E->,
 <music21.chord.Chord B- E->,
 <music21.note.Note E->,
 <music21.chord.Chord G# C>,
 <music21.note.Note E->,
 <music21.chord.Chord F# G# C>,
 <music21.chord.Chord B- E->,
 <music21.chord.Chord B- E->,
 <music21.note.Note E->,
 <music21.chord.Chord F# G# C>,
 <music21.note.Note C#>,
 <music21.chord.Chord B- E->,
 <music21.note.Note E->,
 <music21.note.Note B->,
 <music21.chord.Chord E F#

In [65]:
# create a stream object from the geneated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi',fp='test_output.midi')

'test_output.midi'

In [66]:
midi_stream.show('midi')

In [67]:
# Pratice --> Add guitar with piano